In [2]:
!git clone https://{token}@github.com/azaveri879/skingpt-evaluation.git
%cd skingpt-evaluation

Cloning into 'skingpt-evaluation'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 109 (delta 20), reused 101 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 1.73 MiB | 26.04 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/skingpt-evaluation


In [3]:
!git pull

Already up to date.


In [4]:
!pip install huggingface_hub
!pip install matplotlib google-cloud-storage

In [5]:
import collections
import copy
import hashlib
import io
import os
import subprocess
import textwrap
import time
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from typing import List, Text

from PIL import Image

import numpy as np
import pandas as pd
import tabulate


import tensorflow as tf

import matplotlib.pyplot as plt


In [14]:
from google.colab import auth

# Authenticate user for access. There will be a popup asking you to sign in with your user and approve access.
auth.authenticate_user()

In [15]:
!ls

environment.yml  notebooks  README.md  src


In [17]:
from src.data_utils import download_scin, download_ham10000

# Download SCIN dataset
download_scin("data/scin")


Saved to data/scin/scin_cases.csv
Saved to data/scin/scin_labels.csv


100%|██████████| 10380/10380 [00:00<00:00, 64905.82it/s]


True

In [29]:
!python data/download_ham10000.py

Setting up Kaggle credentials...

Please follow these steps to set up Kaggle credentials:
1. Go to https://www.kaggle.com/account
2. Scroll down to the 'API' section
3. Click 'Create New API Token'
4. This will download a kaggle.json file

Then, please provide the path to your kaggle.json file:
.kaggle/kaggle.json
Kaggle credentials setup complete!
Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
data/ham10000
Organizing dataset files...
Dataset downloaded and organized in data/ham10000
Total images: 10015


In [30]:
ham_dir = "data/ham10000"
metadata = pd.read_csv(os.path.join(ham_dir, "HAM10000_metadata.csv"))
image_folder = os.path.join(ham_dir, "images")

In [31]:
# For SCIN
scin_dir = "data/scin"
scin_metadata = pd.read_csv(os.path.join(scin_dir, "scin_merged.csv"))
scin_image_folder = os.path.join(scin_dir, "images")

In [32]:
from src.data_utils import prepare_dataset

# For HAM10000
train_df, val_df, test_df = prepare_dataset(ham_dir, "ham10000")

# For SCIN
train_df, val_df, test_df = prepare_dataset(scin_dir, "scin")

In [47]:
!pip install iopath
!pip install webdataset
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 124.5 MB/s eta 0:00:00


In [56]:
from huggingface_hub import notebook_login
notebook_login()

In [58]:
!git clone https://huggingface.co/meta-llama/Llama-2-13b-chat-hf

fatal: destination path 'Llama-2-13b-chat-hf' already exists and is not an empty directory.


In [61]:
from skingpt4.common.config import Config
from skingpt4.common.registry import registry

import argparse

args = argparse.Namespace(
    cfg_path="eval_configs/skingpt4_eval_llama2_13bchat.yaml",
    options=[]
)
cfg = Config(args)
model_config = cfg.model_cfg
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config)

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [48]:
from skingpt4.models.skin_gpt4 import skingpt4

In [54]:
model = skingpt4(llm_model="meta-llama/Llama-2-7b-hf")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

100%|██████████| 1.89G/1.89G [00:44<00:00, 45.2MB/s]
BertLMHeadModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading VIT Done
Loading Q-Former


100%|██████████| 413M/413M [00:02<00:00, 199MB/s]


Loading Q-Former Done
Loading LLM tokenizer


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: ''.

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class HAM10000Dataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['image_id'] + '.jpg')
        image = Image.open(img_path).convert('RGB')
        label = row['dx']  # adjust as needed
        if self.transform:
            image = self.transform(image)
        return image, label

# Example DataLoader
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
train_dataset = HAM10000Dataset(train_df, os.path.join(ham_dir, "images"), transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [49]:
!git add *

The following paths are ignored by one of your .gitignore files:
data
hint: Use -f if you really want to add them.
hint: Turn this message off by running
hint: "git config advice.addIgnoredFile false"


In [51]:
!git config --global user.email "zaveri.a@husky.neu.edu"
!git config --global user.name "Aakash Zaveri"

In [52]:
!git commit -m "colab"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [53]:
!git push

Everything up-to-date
